# Unit 1: Navigation Indoor

<img src="img/jackal_clearpath_instruction_manual_unit1_intro.png"/>

<img src="img/Clearpath-Logo.png" width="300"/>

<img src="img/robotignite_logo_text.png" width="300"/>

So in this unit you will learn how to create your own navigation script, generate maps and save them.

The first thing you will need for a robot that patroles is the ability to move around without crashing into everything. For this you will learn:<br>
* How to move the robot
* How to navigate without map.
* How to map the environment
* How to set the navigation stack to be able to send him a location and be able to reach that destination.
* How to send it a sequence of waypoints and execute those movements.

## How to Move the Robot

To move Jackal, as you saw in the introduction you can do it many ways, but in reality all do the same thing: **publish into the /cmd_vel topic**.

<p style="background:#EE9023;color:white;">Exercise U1-1</p>

Publish through the web shell in the mentioned topic and see if you can make Jackal move forwards, backwards, left, right and in circles.

<p style="background:#EE9023;color:white;">END Exercise U1-1</p>

You should have done something like this:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rostopic pub /cmd_vel geometry_msgs/Twist [TAB][TAB]
</th>
</tr>
</table>

With this command you can then fill in the structure that appears after hitting TAB key two consecutive times, to publish the message.

<p style="background:#EE9023;color:white;">Exercise U1-2</p>

Create a package that you  will use along the rest of this course where you can have all your programs. Use the name **my_jackal_tools** for future refference.<br>
Then create a program that moves jackal around publishing in the **/cmd_vel** topic. See how it moves and its limitations. This is crucial to know what you can do with your robot.<br>

<p style="background:#EE9023;color:white;">END Exercise U1-2</p>

## How to navigate without map

Here you will learn how to reproduce the system that you launched in the Demo to make Jackal navigate only with the map generated localy by the detection of obstacles.

Here we will go over the files and only point out particular elements for Jackal. If you want to know more details on how navigation is performed in ROS, please do consider to do **RobotIgnites Navigation in ROS Course** before continuing.

### Create the launch file for move base

This is the core of how to make Jackal move around. The package <a href="http://wiki.ros.org/move_base">move_base</a> makes all the path planning for you.<br>
You will use **move_base** for all the navigation, but here it only has Jackals sensors to detect obstacles.<br>
Here you have an example of how **move_base** should be launched.

**odom_navigation.launch**

In [ ]:
<launch>

 <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">

  <rosparam file="$(find my_jackal_tools)/params/costmap_common_params.yaml" command="load" ns="global_costmap" />
  <rosparam file="$(find my_jackal_tools)/params/costmap_common_params.yaml" command="load" ns="local_costmap" />

  <rosparam file="$(find my_jackal_tools)/params/odom_nav_params/global_costmap_params.yaml" command="load" />
  <rosparam file="$(find my_jackal_tools)/params/odom_nav_params/local_costmap_params.yaml" command="load" />

  <rosparam file="$(find my_jackal_tools)/params/base_local_planner_params.yaml" command="load" />
  <rosparam file="$(find my_jackal_tools)/params/move_base_params.yaml" command="load" />

  <param name="base_global_planner" type="string" value="navfn/NavfnROS" />
  <param name="base_local_planner" value="base_local_planner/TrajectoryPlannerROS"/>

  <remap from="odom" to="odometry/filtered" />
 </node>

</launch>

Here we have some elements to comment on:

Basically **move_base** takes in configuration files of .yaml format that define everything. From which **planner** its going to use, to how the maps are used or sensors.

**costmap_common_params.yaml**

In [ ]:
map_type: costmap
origin_z: 0.0
z_resolution: 1
z_voxels: 2

obstacle_range: 2.5
raytrace_range: 3.0

publish_voxel_map: false
transform_tolerance: 0.5
meter_scoring: true

footprint: [[-0.21, -0.165], [-0.21, 0.165], [0.21, 0.165], [0.21, -0.165]]
footprint_padding: 0.1

plugins:
- {name: obstacles_layer, type: "costmap_2d::ObstacleLayer"}
- {name: inflater_layer, type: "costmap_2d::InflationLayer"}

obstacles_layer:
  observation_sources: scan
  scan: {sensor_frame: front_laser, data_type: LaserScan, topic: front/scan, marking: true, clearing: true, min_obstacle_height: -2.0, max_obstacle_height: 2.0, obstacle_range: 2.5, raytrace_range: 3.0}

inflater_layer:
 inflation_radius: 0.30


Just note the **obstacles_layer**, that uses the topic **/front/scan** to read the laser readings. The camera is not used for navigation.<br>
Also note the ray trace range is only 3 meters , when its <a href="https://www.sick.com/media/pdf/2/42/842/dataSheet_LMS111-10100_1041114_en.pdf">SICK laser</a> has a range up to 20 meters. This is just to make detections faster and not have in mind areas that arent close enough.

**global_costmap_params.yaml**

In [ ]:
global_costmap:
  global_frame: odom
  robot_base_frame: base_link
  update_frequency: 20
  publish_frequency: 5
  width: 40.0
  height: 40.0
  resolution: 0.05
  origin_x: -20.0
  origin_y: -20.0
  static_map: true
  rolling_window: false

**local_costmap_params.yaml**

In [ ]:
local_costmap:
   global_frame: odom
   robot_base_frame: base_link
   update_frequency: 20.0
   publish_frequency: 5.0
   width: 10.0
   height: 10.0
   resolution: 0.05
   static_map: false
   rolling_window: true

**base_local_planner_params**

In [ ]:
TrajectoryPlannerROS:

  # Robot Configuration Parameters
  acc_lim_x: 10.0
  acc_lim_theta:  20.0

  max_vel_x: 0.5
  min_vel_x: 0.1

  max_vel_theta: 1.57
  min_vel_theta: -1.57
  min_in_place_vel_theta: 0.314

  holonomic_robot: false
  escape_vel: -0.5

  # Goal Tolerance Parameters
  yaw_goal_tolerance: 0.157
  xy_goal_tolerance: 0.25
  latch_xy_goal_tolerance: false

  # Forward Simulation Parameters
  sim_time: 2.0
  sim_granularity: 0.02
  angular_sim_granularity: 0.02
  vx_samples: 6
  vtheta_samples: 20
  controller_frequency: 20.0

  # Trajectory scoring parameters
  meter_scoring: true # Whether the gdist_scale and pdist_scale parameters should assume that goal_distance and path_distance are expressed in units of meters or cells. Cells are assumed by default (false).
  occdist_scale:  0.1 #The weighting for how much the controller should attempt to avoid obstacles. default 0.01
  pdist_scale: 0.75  #     The weighting for how much the controller should stay close to the path it was given . default 0.6
  gdist_scale: 1.0 #     The weighting for how much the controller should attempt to reach its local goal, also controls speed  default 0.8

  heading_lookahead: 0.325  #How far to look ahead in meters when scoring different in-place-rotation trajectories
  heading_scoring: false  #Whether to score based on the robot's heading to the path or its distance from the path. default false
  heading_scoring_timestep: 0.8   #How far to look ahead in time in seconds along the simulated trajectory when using heading scoring (double, default: 0.8)
  dwa: true #Whether to use the Dynamic Window Approach (DWA)_ or whether to use Trajectory Rollout
  simple_attractor: false
  publish_cost_grid_pc: true

  #Oscillation Prevention Parameters
  oscillation_reset_dist: 0.05 #How far the robot must travel in meters before oscillation flags are reset (double, default: 0.05)
  escape_reset_dist: 0.1
  escape_reset_theta: 0.1


Note some parameters like:
* holonomic_robot: Jackal is non holonomic
* yaw_goal_tolerance and xy_goal_tolerance are relatively low.

**move_base_params.yaml**

In [ ]:
shutdown_costmaps: false

controller_frequency: 20.0
controller_patience: 15.0

planner_frequency: 20.0
planner_patience: 5.0

oscillation_timeout: 0.0
oscillation_distance: 0.5

recovery_behavior_enabled: true
clearing_rotation_allowed: true

Here the **clearing_rotation_allowed** which meand that it will turn around to clear space if for some reason it gets lost to try and recover itself. Its also usefull for situations when a moving obstacle was detected in its path. By turning it gives time to the obstacle to move out the way and therefore clear the localcostmap.

For **localPlanner** the **TrajectoryPlannerROS** is used. 

For **globalPlanner** the **navfn/NavfnROS** is used. 

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

* Inside the new **my_jackal_tools** create the files given examples and the launch file **odom_navigation.launch**.
* Then launch rviz through the command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun rviz rviz
</th>
</tr>
</table>

* Add the RVIZ elements: 

<img src="img/jackal_clearpath_instruction_manual_unit1_rvizelements_odomnav.png"/>

* Note the two paths, one for the local and another for the global planner.
* One map only for the **LocalCostmap**. In this case both local and global maps are the same.
* **move_base_simple/goal** will be the value of the topic for the **Pose**

Remember to save the configuration of RVIZ when finished to use it afterwards. Name it **navigation** and save it in a folder called **rviz**.

<p style="background:#EE9023;color:white;">END Exercise U1-3</p>

You should get the same result as in the Demo 

<img src="img/jackal_clearpath_instruction_manual_unit0_nav_odom.png">

## Navigate with a MAP

Now you are going to learn how to navigate with a map. But first you have to generate that map:

### Create a Map

To create a Map you need to launch two elements: **GMapping** and **MoveBase**.
* **GMapping**: in the one in charge of generating and saving the map. More info <a href="http://wiki.ros.org/gmapping">here</a>.
* **MoveBase**: The one in charge of sending the commands to /cmd_vel to move the robot around based on sensors and now generating map.

So you can create a launch file that launches both:

**start_mapping.launch**

In [ ]:
<launch>

  <!--- Run gmapping -->
  <include file="$(find my_jackal_tools)/launch/gmapping.launch" />

  <!--- Run Move Base -->
  <include file="$(find my_jackal_tools)/launch/with_map_move_base.launch" />

</launch>


So first, take a look at the **gmapping.launch**. It should be something similar to this:

In [ ]:
<launch>

  <arg name="scan_topic" default="front/scan" />

  <node pkg="gmapping" type="slam_gmapping" name="slam_gmapping" output="screen">

    <param name="odom_frame" value="odom"/>
    <param name="base_frame" value="base_link"/>
    <param name="map_frame" value="map"/>

    <!-- Process 1 out of every this many scans (set it to a higher number to skip more scans)  -->
    <param name="throttle_scans" value="1"/>

    <param name="map_update_interval" value="5.0"/> <!-- default: 5.0 -->

    <!-- The maximum usable range of the laser. A beam is cropped to this value.  -->
    <param name="maxUrange" value="5.0"/>

    <!-- The maximum range of the sensor. If regions with no obstacles within the range of the sensor should appear as free space in the map, set maxUrange < maximum range of the real sensor <= maxRange -->
    <param name="maxRange" value="10.0"/>

    <param name="sigma" value="0.05"/>
    <param name="kernelSize" value="1"/>
    <param name="lstep" value="0.05"/>
    <param name="astep" value="0.05"/>
    <param name="iterations" value="5"/>
    <param name="lsigma" value="0.075"/>
    <param name="ogain" value="3.0"/>
    <param name="minimumScore" value="0.0"/>
    <!-- Number of beams to skip in each scan. -->
    <param name="lskip" value="0"/>

    <param name="srr" value="0.01"/>
    <param name="srt" value="0.02"/>
    <param name="str" value="0.01"/>
    <param name="stt" value="0.02"/>

    <!-- Process a scan each time the robot translates this far  -->
    <param name="linearUpdate" value="0.1"/>

    <!-- Process a scan each time the robot rotates this far  -->
    <param name="angularUpdate" value="0.05"/>

    <param name="temporalUpdate" value="-1.0"/>
    <param name="resampleThreshold" value="0.5"/>

    <!-- Number of particles in the filter. default 30        -->
    <param name="particles" value="10"/>

<!-- Initial map size  -->
    <param name="xmin" value="-10.0"/>
    <param name="ymin" value="-10.0"/>
    <param name="xmax" value="10.0"/>
    <param name="ymax" value="10.0"/>

    <!-- Processing parameters (resolution of the map)  -->
    <param name="delta" value="0.02"/>

    <param name="llsamplerange" value="0.01"/>
    <param name="llsamplestep" value="0.01"/>
    <param name="lasamplerange" value="0.005"/>
    <param name="lasamplestep" value="0.005"/>

    <remap from="scan" to="$(arg scan_topic)"/>
  </node>
</launch>


Here are basically all the pareameters related to the map generation defined.<br>
Note the maxUrange = 5.0 meters and maxRange = 10 meters. As you see here the laser is also cut of to emprove performance.

**with_map_move_base.launch**

In [ ]:
<launch>
 
  <node pkg="move_base" type="move_base" respawn="false" name="move_base" output="screen">

    <rosparam file="$(find my_jackal_tools)/params/costmap_common_params.yaml" command="load" ns="global_costmap" />
    <rosparam file="$(find my_jackal_tools)/params/costmap_common_params.yaml" command="load" ns="local_costmap" />
    
    <rosparam file="$(find my_jackal_tools)/params/map_nav_params/local_costmap_params.yaml" command="load" />
    <rosparam file="$(find my_jackal_tools)/params/map_nav_params/global_costmap_params.yaml" command="load" />
    
    <rosparam file="$(find my_jackal_tools)/params/base_local_planner_params.yaml" command="load" />
    <rosparam file="$(find my_jackal_tools)/params/move_base_params.yaml" command="load" />
    
    <param name="base_global_planner" type="string" value="navfn/NavfnROS" />
    <param name="base_local_planner" value="base_local_planner/TrajectoryPlannerROS"/>
    
    <remap from="odom" to="odometry/filtered" />
  </node>

</launch>

**odom_navigation.launch**

Here you can see there are some differences with the **odom_navigation.launch**.<br>

In [ ]:
<rosparam file="$(find my_jackal_tools)/params/map_nav_params/local_costmap_params.yaml" command="load" />
<rosparam file="$(find my_jackal_tools)/params/map_nav_params/global_costmap_params.yaml" command="load" />

*****************

<rosparam file="$(find my_jackal_tools)/params/odom_nav_params/global_costmap_params.yaml" command="load" />
<rosparam file="$(find my_jackal_tools)/params/odom_nav_params/local_costmap_params.yaml" command="load" />

The **only** difference is in the **global and local costmaps**. Which is understandable because outside that, the res is only related to move base internals and sensors. These are the only ones that talk about maps.

**local_costmap_params.yaml**

In [ ]:
local_costmap:
   global_frame: map
   robot_base_frame: base_link
   update_frequency: 20.0
   publish_frequency: 5.0
   width: 10.0
   height: 10.0
   resolution: 0.05
   static_map: false
   rolling_window: true

In [ ]:
global_frame: map
    
***

global_frame: odom

The only difference is that you are changing of **global frame**. Now you calculate all the paths based on the map frame.

**global_costmap_params.yaml**

In [ ]:
global_costmap:
   global_frame: map
   robot_base_frame: base_link
   update_frequency: 20.0
   publish_frequency: 5.0
   width: 40.0
   height: 40.0
   resolution: 0.05
   origin_x: -20.0
   origin_y: -20.0
   static_map: true
   rolling_window: false

   plugins:
   - {name: static_layer, type: "costmap_2d::StaticLayer"}
   - {name: obstacles_layer, type: "costmap_2d::ObstacleLayer"}
   - {name: inflater_layer, type: "costmap_2d::InflationLayer"}

In [ ]:
global_frame: map

plugins:
- {name: static_layer, type: "costmap_2d::StaticLayer"}
- {name: obstacles_layer, type: "costmap_2d::ObstacleLayer"}
- {name: inflater_layer, type: "costmap_2d::InflationLayer"}

****

global_frame: odom

The only differences are:
* The Global frame used.
* Use of plugins: These are plugins that their only function is to generate the costmaps. See <a href="http://wiki.ros.org/costmap_2d">here</a> for more infomation .

<p style="background:#EE9023;color:white;">Exercise U1-4</p>

* Inside the new **my_jackal_tools** create the files given examples and the launch file **start_mapping.launch**.
* Then launch rviz through the command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun rviz rviz
</th>
</tr>
</table>

* Add the RVIZ elements: 

<img src="img/jackal_clearpath_instruction_manual_unit1_rvizelements_gmappingnav.png"/>

* Note the two Maps, one reads the **/map** topic. The Other the **GlobalCostmap**.<br>
The other values are the same as the No map configuration.

Remember to save the configuration of RVIZ when finished to use it afterwards. Name it **gmapping** and save it in a folder called **rviz**.

<p style="background:#EE9023;color:white;">END Exercise U1-4</p>

You should get smething similar to this if everything is ok:

<img src="img/jackal_clearpath_instruction_manual_unit1_gmapping1.png">

You see that instantly it started to generate the map. No its time to move Jackal around.<br>
You can move it through the **start_teleop.launch** used in the Demo, by the **InteractiveMarkers** or better: **through poses in the map**!<br>
This works because you are navigating in the same way as you were in the **No Map navigation**. So you can use 2D Poses to generate the map.<br>
Just **Be Carefull** not to give very far away poses, because remmember you dont have a map and the path planning is not as good.

<img src="img/jackal_clearpath_instruction_manual_unit1_mapcreation.gif">

Once you have all the map as you would like, then you have to save it to be able to use it for navigation.<br>
For this you have to execute the following command where you want the map files to be saved. You have a sugesstion on where to put it but its up to you:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roscd my_jackal_tools<br>
mkdir maps<br>
cd maps<br>
rosrun map_server map_saver -f mymap
</th>
</tr>
</table>

This should have generated two files: **mymap.pgm and mymap.yaml**.<br>
You will use the **pgm** image for cleaning the map generated, and the **yaml** is the file directly used by the PathPLanning afterwards. It also contains valueble information like the path to the image of the maps dimmensions.

**mymap.yaml**

In [ ]:
image: new_clean_map.pgm
resolution: 0.050000
origin: [-50.000000, -50.000000, 0.000000]
negate: 0
occupied_thresh: 0.65
free_thresh: 0.196



As you can see here, the file **mymap.pgm** is in the same location as the **mymap.yaml**. Thats why the imgae variable only has the name of the image. But here can go a full absolute path.

### Navigate with your map

And finally you now have to launch the package **amcl** to localise Jackal with the map. For that you create the following launch:

**start_navigation_with_map.launch**

In [ ]:
<launch>

    <!-- Run the map server -->
    <arg name="map_file" default="$(find jackal_tools)/maps/mymap.yaml"/>
    <node name="map_server" pkg="map_server" type="map_server" args="$(arg map_file)" />
    
    <!--- Run AMCL -->
    <include file="$(find jackal_tools)/launch/amcl.launch" />
    
    <!--- Run Move Base -->
    <include file="$(find jackal_tools)/launch/with_map_move_base.launch" />

</launch>

Here there three main parts:
* **map_server**: The Node that publishes the map saved.
* **amcl**: The node that localises Jackal based on map readings
* **move_base**: This move base has the configuration of moving with a map.

So really there is only one launch file to comment on which is:

**amcl.launch**

In [ ]:
<launch>

  <arg name="use_map_topic" default="false"/>
  <arg name="scan_topic" default="front/scan" />

  <node pkg="amcl" type="amcl" name="amcl">
    <param name="use_map_topic" value="$(arg use_map_topic)"/>
    <!-- Publish scans from best pose at a max of 10 Hz -->
    <param name="odom_model_type" value="diff"/>
    <param name="odom_alpha5" value="0.1"/>
    <param name="gui_publish_rate" value="10.0"/>
    <param name="laser_max_beams" value="720"/>
    <param name="laser_min_range" value="0.1"/>
    <param name="laser_max_range" value="30.0"/>
    <param name="min_particles" value="500"/>
    <param name="max_particles" value="2000"/>
    <!-- Maximum error between the true distribution and the estimated distribution. -->
    <param name="kld_err" value="0.05"/>
    <param name="kld_z" value="0.99"/>
    <param name="odom_alpha1" value="0.2"/>
    <param name="odom_alpha2" value="0.2"/>
    <!-- translation std dev, m -->
    <param name="odom_alpha3" value="0.2"/>
    <param name="odom_alpha4" value="0.2"/>
    <param name="laser_z_hit" value="0.5"/>
    <param name="laser_z_short" value="0.05"/>
    <param name="laser_z_max" value="0.05"/>
    <param name="laser_z_rand" value="0.5"/>
    <param name="laser_sigma_hit" value="0.2"/>
    <param name="laser_lambda_short" value="0.1"/>
    <param name="laser_model_type" value="likelihood_field"/>
    <!-- Maximum distance to do obstacle inflation on map, for use in likelihood_field model. -->
    <param name="laser_likelihood_max_dist" value="2.0"/>
    <!-- Translational movement required before performing a filter update.  -->
    <param name="update_min_d" value="0.1"/>
    <!--Rotational movement required before performing a filter update. -->
    <param name="update_min_a" value="0.314"/>
    <param name="odom_frame_id" value="odom"/>
    <param name="base_frame_id" value="base_link"/>
    <param name="global_frame_id" value="map"/>
    <!-- Number of filter updates required before resampling. -->
    <param name="resample_interval" value="1"/>
    <!-- Increase tolerance because the computer can get quite busy -->
    <param name="transform_tolerance" value="1.0"/>
    <!-- Exponential decay rate for the slow average weight filter, used in deciding when to recover by adding random poses. A good value might be 0.001. -->
    <param name="recovery_alpha_slow" value="0.0"/>
    <!--Exponential decay rate for the fast average weight filter, used in deciding when to recover by adding random poses. A good value might be 0.1. -->
    <param name="recovery_alpha_fast" value="0.1"/>
    <!-- Initial pose mean -->
    <param name="initial_pose_x" value="0.0" />
    <param name="initial_pose_y" value="0.0" />
    <param name="initial_pose_a" value="0.0" />
    <!-- When set to true, AMCL will subscribe to the map topic rather than making a service call to receive its map.-->
    <param name="receive_map_topic" value="true"/>
    <!--  When set to true, AMCL will only use the first map it subscribes to, rather than updating each time a new one is received. -->
    <param name="first_map_only" value="false"/>
    <remap from="scan" to="$(arg scan_topic)"/>
  </node>

</launch>


<p style="background:#EE9023;color:white;">Exercise U1-5</p>

* Inside the new **my_jackal_tools** create the files given examples and the launch file **start_navigation_with_map.launch**.
* Then launch rviz through the command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
rosrun rviz rviz
</th>
</tr>
</table>

* Add the RVIZ elements: 

<img src="img/jackal_clearpath_instruction_manual_unit1_rvizelements_navwithmap.png"/>

* Note the two Paths, **local** and **global**<br>
* Note the particle Cloud, that indicates where Jackal thinks he is. Its reading from the topic **/particlecloud**

Remember to save the configuration of RVIZ when finished to use it afterwards. Name it **lacalization** and save it in a folder called **rviz**.

<p style="background:#EE9023;color:white;">END Exercise U1-5</p>

When launching this could happen:

<img src="img/jackal_clearpath_instruction_manual_unit1_errorinestimate.png"/>

This is because Jackal doesn t know where it is initialy, so you have to indicate the position and orientation with the **2D Estimate Pose** icon. Once done, it should navigate without problems. It will localize itself after a few seconds of movement. You know it is well localised because all the arrows are conscentrated into the position of the robot.

<img src="img/jackal_clearpath_instruction_manual_unit1_finalnav.gif"/>

## Add WayPoints

So there you have it. Now your Jackal can navigate around one position at a time. But what if you want to generate in an easy way a sequence of waypoints so that it patroles around the **main interest points of the area**?

You are going to use a ROS package called **follow_waypoints**, you can get more information http://wiki.ros.org/follow_waypoints.
This package basically tracks the **Estimate pose** that you place in RVIZ and stores it. Then when you have finished you just have to publish in a topic that starts sending those positions to the movebase system.

### Steps to do it:

The first step is to start the **start_navigation_with_map.launch** if you don't have it running:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roslaunch my_jackal_tools start_navigation_with_map.launch
</th>
</tr>
</table>

The Second step is to start the waypoint server. This server will listen to publications into the topic **/initialpose** and store those poses until its instructed to send them to **move_base** to be executed:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
roslaunch follow_waypoints follow_waypoints.launch
</th>
</tr>
</table>

Then you have to start RVIZ, and add a **PoseArray** element, susbcribed to the topic **/waypoints**, and change its name to WayPoints. This will now show all the waypoints you set from now on.

You can change the colour also if you want. In this example its changed to **green** to see it better and differenciate from the posearray of **psoition estimates**.

Now you have to set the waypoints. For this you have to select the **PoseEstimate** and set it where you want it. Because this will move the estimated pose of the robot around its important that **the last waypoint is where the robot is right now**. Otherwise it will be harder for the robot to get to the place. 

<img src="img/jackal_clearpath_instruction_manual_unit1_waypoints2.gif"></img>

You should get something similar to this:

<img src="img/jackal_clearpath_instruction_manual_unit1_waypoints1.png" width="600"></img>

You should also get in webshell where you launched the waypoint server a message stating that it recieved the waypoint:

In [ ]:
[INFO][139816504842048][/follow_waypoints/execute:92]: Waiting to recieve waypoints via Pose msg on topic /initialpose         
[INFO][139816504842048][/follow_waypoints/execute:93]: To start following waypoints: 'rostopic pub /path_ready std_msgs/Empty -
1'                                                                                                                             
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint                                                  
[INFO][139816504842048][/follow_waypoints/execute:104]: Recieved new waypoint

Now you just have to publish in the topic **/path_ready** to start sending waypoints to the movebase.

If you want to see the global path generated just activate it in RVIZ.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p><br>
rostopic pub /path_ready std_msgs/Empty -1
</th>
</tr>
</table>

You should get something similar to this for each waypoint executed. Bare in mind that because you move the robot around, it will probably first do a recovery behaviour to clean up all the false obstacles generated through changing the estipate pose:

<img src="img/jackal_clearpath_instruction_manual_unit1_waypoints3.gif"></img>

After the recovery behaviour will come the execution of each way point:

<img src="img/jackal_clearpath_instruction_manual_unit1_waypoints4.gif"></img>

As you can see, its a nice way to do it , but its a bit cumbersome and messy. Specially because it has to change the initial position of the robot which is not very desirable. This is because this waypoint manager listenes to the topic **/initialpose**. Th eproblem is that **amlc localiser** also. Thats why when changed, the robot localises itself also in another place.<br>
So the next step is creating your own version. So you will part from the code that you just used that is installed in the system and is courtsesy of <a href="https://github.com/danielsnider">Daniel Snider</a> ,who created the original Waypoints manager <a href="https://github.com/danielsnider/follow_waypoints">Program</a>.

In order to avoid touching the **/initialpose**, the topic that the **follow_points** listens to, this has to be changed. Not only that put it has to be configurable. See here the final code:

**custom_follow_waypoints.py**

In [ ]:
#!/usr/bin/env python

import threading
import rospy
import actionlib

from smach import State,StateMachine
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import PoseWithCovarianceStamped, PoseArray
from std_msgs.msg import Empty

waypoints = []

class FollowPath(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'], input_keys=['waypoints'])
        self.frame_id = rospy.get_param('~goal_frame_id','map')
        # Get a move_base action client
        self.client = actionlib.SimpleActionClient('move_base', MoveBaseAction)
        rospy.loginfo('Connecting to move_base...')
        self.client.wait_for_server()
        rospy.loginfo('Connected to move_base.')

    def execute(self, userdata):
        global waypoints
        # Execute waypoints each in sequence
        for waypoint in waypoints:
            # Break if preempted
            if waypoints == []:
                rospy.loginfo('The waypoint queue has been reset.')
                break
            # Otherwise publish next waypoint as goal
            goal = MoveBaseGoal()
            goal.target_pose.header.frame_id = self.frame_id
            goal.target_pose.pose.position = waypoint.pose.pose.position
            goal.target_pose.pose.orientation = waypoint.pose.pose.orientation
            rospy.loginfo('Executing move_base goal to position (x,y): %s, %s' %
                    (waypoint.pose.pose.position.x, waypoint.pose.pose.position.y))
            rospy.loginfo("To cancel the goal: 'rostopic pub -1 /move_base/cancel actionlib_msgs/GoalID -- {}'")
            self.client.send_goal(goal)
            self.client.wait_for_result()
        return 'success'

def convert_PoseWithCovArray_to_PoseArray(waypoints):
    """Used to publish waypoints as pose array so that you can see them in rviz, etc."""
    poses = PoseArray()
    poses.header.frame_id = 'map'
    poses.poses = [pose.pose.pose for pose in waypoints]
    return poses

class GetPath(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'], input_keys=['waypoints'], output_keys=['waypoints'])
        # Create publsher to publish waypoints as pose array so that you can see them in rviz, etc.
        self.poseArray_publisher = rospy.Publisher('/waypoints', PoseArray, queue_size=1)
        self._custom_waypointstopic = rospy.get_param('~custom_waypointstopic','/my_waypoints_list')
        # Start thread to listen for reset messages to clear the waypoint queue
        def wait_for_path_reset():
            """thread worker function"""
            global waypoints
            while not rospy.is_shutdown():
                data = rospy.wait_for_message('/path_reset', Empty)
                rospy.loginfo('Recieved path RESET message')
                self.initialize_path_queue()
                rospy.sleep(3) # Wait 3 seconds because `rostopic echo` latches
                               # for three seconds and wait_for_message() in a
                               # loop will see it again.
        reset_thread = threading.Thread(target=wait_for_path_reset)
        reset_thread.start()

    def initialize_path_queue(self):
        global waypoints
        waypoints = [] # the waypoint queue
        # publish empty waypoint queue as pose array so that you can see them the change in rviz, etc.
        self.poseArray_publisher.publish(convert_PoseWithCovArray_to_PoseArray(waypoints))

    def execute(self, userdata):
        global waypoints
        self.initialize_path_queue()
        self.path_ready = False

        # Start thread to listen for when the path is ready (this function will end then)
        def wait_for_path_ready():
            """thread worker function"""
            data = rospy.wait_for_message('/path_ready', Empty)
            rospy.loginfo('Recieved path READY message')
            self.path_ready = True
        ready_thread = threading.Thread(target=wait_for_path_ready)
        ready_thread.start()

        topic = self._custom_waypointstopic
        rospy.loginfo("Waiting to recieve waypoints via Pose msg on topic %s" % topic)
        rospy.loginfo("To start following waypoints: 'rostopic pub /path_ready std_msgs/Empty -1'")

        # Wait for published waypoints
        while not self.path_ready:
            try:
                pose = rospy.wait_for_message(topic, PoseWithCovarianceStamped, timeout=1)
            except rospy.ROSException as e:
                if 'timeout exceeded' in e.message:
                    continue  # no new waypoint within timeout, looping...
                else:
                    raise e
            rospy.loginfo("Recieved new waypoint")
            waypoints.append(pose)
            # publish waypoint queue as pose array so that you can see them in rviz, etc.
            self.poseArray_publisher.publish(convert_PoseWithCovArray_to_PoseArray(waypoints))

        # Path is ready! return success and move on to the next state (FOLLOW_PATH)
        return 'success'

class PathComplete(State):
    def __init__(self):
        State.__init__(self, outcomes=['success'])

    def execute(self, userdata):
        rospy.loginfo('###############################')
        rospy.loginfo('##### REACHED FINISH GATE #####')
        rospy.loginfo('###############################')
        return 'success'

def main():
    rospy.init_node('custom_follow_waypoints')

    sm = StateMachine(outcomes=['success'])

    with sm:
        StateMachine.add('GET_PATH', GetPath(),
                           transitions={'success':'FOLLOW_PATH'},
                           remapping={'waypoints':'waypoints'})
        StateMachine.add('FOLLOW_PATH', FollowPath(),
                           transitions={'success':'PATH_COMPLETE'},
                           remapping={'waypoints':'waypoints'})
        StateMachine.add('PATH_COMPLETE', PathComplete(),
                           transitions={'success':'GET_PATH'})

    outcome = sm.execute()

if __name__ == "__main__":
    main()

The new topic is retrived through a parameter setting.

In [ ]:
self._custom_waypointstopic = rospy.get_param('~custom_waypointstopic','/my_waypoints_list')

This means that when this node is launched, it will have to be set this parameter if we don't want it to have its default value **/my_waypoints_list**.<br>
Here you have an example of how it should be launched:

**start_follow_waypoints.launch**

In [ ]:
<launch>
    <env name="ROSCONSOLE_FORMAT" value="[${severity}][${thread}][${node}/${function}:${line}]: ${message}"/>
     <arg name="waypoints_topic" default="/my_jackal_waypoints"/>
    
    <node pkg="my_jackal_tools" type="custom_follow_waypoints.py" name="custom_follow_waypoints" output="screen" clear_params="true">
        <param name="goal_frame_id" value="map"/>
        <param name="custom_waypointstopic" value="$(arg waypoints_topic)"/>
    </node>

</launch>

Lets comment on various elements here:

In [ ]:
<env name="ROSCONSOLE_FORMAT" value="[${severity}][${thread}][${node}/${function}:${line}]: ${message}"/>

This line changes the way the logs are shown, in this case it will show which node, function and line in the code is that log executed.

In [ ]:
<arg name="waypoints_topic" default="/my_jackal_waypoints"/>

Here you state which topic where you will have to publish the waypoint.

In [ ]:
<param name="goal_frame_id" value="map"/>
<param name="custom_waypointstopic" value="$(arg waypoints_topic)"/>

Here you state which is the frame and the topic params values.

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

Now create a python script that generates those waypoints and when finished executes the moving system to follow those waypoints.<br>
To do so, you will need to publish directly into the topic **/my_jackal_waypoints** or the value that you stated in tha launch file **start_follow_waypoints.launch**.<br>
Then you will have to publish to start in the topic **/path_ready**.<br>
Tasks:
* Make it generate circular paths
* Eight shaped paths
* Dynamic waypoints based on the position of the standing person. For this last one you will need to acces the TF data of the standingPerson.

<p style="background:#EE9023;color:white;">Exercise U1-3</p>

<p style="background:green;color:white;">Solution Exercise U.3</p>

Please Try to do it by yourself unless you get stuck or need some inspiration. You will learn much more if you fight for each exercise.

<img src="img/robotignite_logo_text.png"/>

You should get something similar to this:

<img src="img/jackal_clearpath_instruction_manual_unit1_waypoints_example.gif"/>

Here you have an example of a publisher that generates a straight line waypoint path and executes it.

**waypoint_publisher.py**

In [ ]:
#!/usr/bin/env python
# license removed for brevity
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import PoseWithCovarianceStamped
import time
import tf
from std_msgs.msg import Empty

"""
user:~$ rosmsg show geometry_msgs/PoseWithCovarianceStamped                                                                                                               
std_msgs/Header header                                                                                                                                                    
  uint32 seq                                                                                                                                                              
  time stamp                                                                                                                                                              
  string frame_id                                                                                                                                                         
geometry_msgs/PoseWithCovariance pose                                                                                                                                     
  geometry_msgs/Pose pose                                                                                                                                                 
    geometry_msgs/Point position                                                                                                                                          
      float64 x                                                                                                                                                           
      float64 y                                                                                                                                                           
      float64 z                                                                                                                                                           
    geometry_msgs/Quaternion orientation                                                                                                                                  
      float64 x                                                                                                                                                           
      float64 y                                                                                                                                                           
      float64 z                                                                                                                                                           
      float64 w                                                                                                                                                           
  float64[36] covariance 
"""

def talker():
    pub_wp = rospy.Publisher('/my_jackal_waypoints', PoseWithCovarianceStamped, queue_size=1)
    
    pub_init_wp = rospy.Publisher('/path_ready', Empty, queue_size=1)
    
    rospy.init_node('waypoint_publisher', anonymous=True)
    rate = rospy.Rate(10) # 10hz
    
    my_wp = PoseWithCovarianceStamped()
    my_wp.header.stamp = rospy.Time.now()
    my_wp.header.frame_id = "/map"
    
    
    init_value = 2.0
    
    roll = 0
    pitch = 0
    yaw = 0.7
    quaternion = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
    #type(pose) = geometry_msgs.msg.Pose
    my_wp.pose.pose.orientation.x = quaternion[0]
    my_wp.pose.pose.orientation.y = quaternion[1]
    my_wp.pose.pose.orientation.z = quaternion[2]
    my_wp.pose.pose.orientation.w = quaternion[3]


    for i in range(5):
        rospy.loginfo(str(i))
        my_wp.pose.pose.position.x = float(i) + init_value 
        my_wp.pose.pose.position.y = float(i) + init_value
        
        while not rospy.is_shutdown():
            connections = pub_wp.get_num_connections()
            if connections > 0:
                pub_wp.publish(my_wp)
                break
            rospy.loginfo("Waiting for /my_jackal_waypoints topic")
            rate.sleep()
        rospy.loginfo("Published waypoint number ="+str(i))
        time.sleep(2)
    
    start_command = Empty()
    
    while not rospy.is_shutdown():
            connections = pub_init_wp.get_num_connections()
            if connections > 0:
                pub_init_wp.publish(start_command)
                rospy.loginfo("Sent waypoint list execution command")
                break
            rospy.loginfo("Waiting for /path_ready topic")
            rate.sleep()


if __name__ == '__main__':
    try:
        talker()
    except rospy.ROSInterruptException:
        pass

### Extra: Launch your RVIZ configs easily

Here you have an example of how to launch RVIZ sessions faster. You just have to create this tinny launch that retrieves the name of the **rviz** configuration file saved in the **rviz** folder. You will use it in the next units for sure.

**view_robot.launch**

In [ ]:
<launch>
  <param name="use_gui" value="true"/>
  <arg name="config" default="complete" />
  <node name="rviz" pkg="rviz" type="rviz" args="-d $(find my_jackal_tools)/rviz/$(arg config).rviz" />
</launch>

And you can use it like so in any launch file:

In [ ]:
<!-- Start RVIZ for Localization -->
<include file="$(find my_jackal_tools)/launch/view_robot.launch">
    <arg name="config" value="localization" />
</include>

This one will load in RVIZ the file called **localization.rviz**.

## Congratulations!! You are now capable of creating your own navigation program for indoor with your Jackal robot.